In [1]:
import numpy as np
import pandas as pd

E:\Software\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
df=pd.read_csv('news.csv')

#Get shape and head
print(df.shape)
df.head()

(6335, 4)


,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [3]:
df.label.value_counts()

REAL    3171
FAKE    3164
Name: label, dtype: int64

In [4]:
labels=df.label
labels.head()

0    FAKE
1    FAKE
2    REAL
3    FAKE
4    REAL
Name: label, dtype: object

In [5]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(df['text'], labels, test_size=0.2, random_state=42)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)

tfidf_train=tfidf_vectorizer.fit_transform(x_train) 
tfidf_test=tfidf_vectorizer.transform(x_test)

E:\Software\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [7]:
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score

kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [8]:
def grid_search(model, grid_param):
    print("Obtaining Best Model for {}".format(model.__class__.__name__))
    grid_search = GridSearchCV(model, grid_param, cv=kf, scoring='accuracy', return_train_score=True, n_jobs=-1)
    grid_search.fit(tfidf_train, y_train)
    print("Best Parameters: ", grid_search.best_params_)
        
    return grid_search.best_estimator_

In [9]:
def performance_measures(model, store_results=True):
    train_acc = cross_val_score(model, tfidf_train, y_train, scoring='accuracy', cv=kf, n_jobs=-1)
    test_acc = cross_val_score(model, tfidf_test, y_test, scoring='accuracy', cv=kf, n_jobs=-1)
    print("Mean Train Accuracy: {}\nMean Test Accuracy: {}".format(train_acc.mean()*100, test_acc.mean()*100))

In [10]:
from sklearn.linear_model import LogisticRegression

logistic_clf = LogisticRegression(solver='lbfgs', penalty='l2', random_state=42, n_jobs=-1)
logistic_param_grid = [{'C':[0.1, 1, 10]}]
best_logistic_clf = grid_search(logistic_clf, logistic_param_grid)

Obtaining Best Model for LogisticRegression
Best Parameters:  {'C': 10}


In [11]:
performance_measures(best_logistic_clf)

Mean Train Accuracy: 93.46884972672808
Mean Test Accuracy: 91.08244374591517


In [14]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(max_features='auto', random_state=42, n_jobs=-1)
forest_param_grid = [{'max_depth':[6, 8, 12],'n_estimators':[50, 100, 200]}]
best_forest_clf = grid_search(forest_clf, forest_param_grid)

Obtaining Best Model for RandomForestClassifier
Best Parameters:  {'max_depth': 12, 'n_estimators': 300}


In [13]:
performance_measures(best_forest_clf)

Mean Train Accuracy: 86.56284864804874
Mean Test Accuracy: 85.08387538514206


In [16]:
from xgboost import XGBClassifier

xgb_clf = XGBClassifier(random_state=42, n_jobs=-1)
xgb_param_grid = [{'max_depth':[3, 8, 12],'n_estimators':[50, 100, 200]}]
best_xgb_clf = grid_search(xgb_clf, xgb_param_grid)

Obtaining Best Model for XGBClassifier
Best Parameters:  {'max_depth': 8, 'n_estimators': 200}


In [17]:
performance_measures(best_xgb_clf)

Mean Train Accuracy: 91.94968369772833
Mean Test Accuracy: 88.0041081821294


In [10]:
from sklearn.pipeline import Pipeline

final_model = Pipeline([('tfidf_vectorizer', tfidf_vectorizer),
                       ('classifier', best_logistic_clf)])

final_model.fit(df.text, labels)

Pipeline(memory=None,
         steps=[('tfidf_vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.7, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words='english', strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('classifier',
                 PassiveAggressiveClassifier(C=1, average=False,
                                             class_weight=N

In [11]:
import joblib

joblib.dump(final_model, 'fake_news_classifier.pkl')

['E:\\StudyMaterials\\Python\\Django\\MyNewspaper\\news\\ml_model\\fake_news_classifier.pkl']